<a href="https://colab.research.google.com/github/Kona-O/topiary/blob/main/alignment_to_ancestors_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alignment to ancestors pipeline

Given an alignment, find the best phylogenetic model, build a maximum-likelihood tree, reconcile this tree with the species tree, and then infer ancestral protein sequences.

In [ ]:
import topiary
from topiary.raxml import RAXML_BINARY
from topiary.generax import GENERAX_BINARY
from topiary._private import installed
from topiary._private import software_requirements
from topiary._private import check
from topiary._private.mpi import check_mpi_configuration
from topiary._private import Supervisor

import os
import random
import string
import shutil
import time

The first code block below initiates the entire aligment_to_ancestors pipeline with default parameters set. Users can change the parameters here. Alternatively, the subsequent code blocks break this pipeline into its separate sections for making step-specific manipulations.

In [ ]:
# Pipeline that starts from an alignment, finds the best phylogenetic model,
# builds a maximum likelihood tree, reconciles this tree with the species tree,
# and then infers ancestral proteins.
alignment_to_ancestors(df,
                      out_dir=None,
                      starting_tree=None,
                      no_bootstrap=False,
                      no_reconcile=False,
                      no_horizontal_transfer=False,
                      alt_cutoff=0.25,
                      model_matrices=["cpREV","Dayhoff","DCMut","DEN","Blosum62",
                                      "FLU","HIVb","HIVw","JTT","JTT-DCMut","LG",
                                      "mtART","mtMAM","mtREV","mtZOA","PMB",
                                      "rtREV","stmtREV","VT","WAG"],
                      model_rates=["","G8"],
                      model_freqs=["","FC","FO"],
                      model_invariant=["","IC","IO"],
                      restart=False,
                      overwrite=False,
                      num_threads=-1,
                      raxml_binary=RAXML_BINARY,
                      generax_binary=GENERAX_BINARY)

### Argument definitions and default parameters for reference:  
---------- 
**df** : *pandas.DataFrame or str*
<br> Topiary data frame or csv written out from topiary df.

**out_dir** : *str, optional*
<br> Output directory. If not specified, create an output directory with the format "alignment_to_ancestors_{randomletters}".

**starting_tree** : *str, optional*
<br> Tree in newick format. This will be used for the best model inference and starting tree for the maximum likelihood tree estimation. If not specified, the maximum parsimony tree is generated and used.

**no_bootstrap** : *bool, default=False*
<br> Do not do bootstrap replicates.

**no_reconcile** : *bool, default=False*
<br> Do not reconcile gene and species trees.

**no_horizontal_transfer** : *bool, default=False*
<br> Whether to allow horizontal transfer during reconciliation. Default is to allow transfer (UndatedDTL; recommended). If flat set, use UndatedDL model.

**alt_cutoff** : *float, default=0.25*
<br> Cutoff to use for altAll alternate ancestral protein sequence generation. Should be between 0 and 1.

**model_matrices** : *list, default=["cpREV","Dayhoff","DCMut","DEN","Blosum62","FLU","HIVb","HIVw","JTT","JTT-DCMut","LG","mtART","mtMAM","mtREV","mtZOA","PMB","rtREV","stmtREV","VT","WAG"]*
<br> List of model matrices to check. If calling from command line, these can be specified directly (:code:`--model_matrices LG ...`) or by specifying a file with models on each line (:code:`--model_matrices SOME_FILE`).

**model_rates** : *list, default=["","G8"]*
<br> Ways to treat model rates. If calling from command line, these can be specified directly (:code:`--model_rates G8 ...`) or by specifying a file with rates on each line (:code:`--model_rates SOME_FILE`).

**model_freqs** : *list, default=["","FC","FO"]*
<br> Ways to treat model freqs. If calling from command line, these can be specified directly (:code:`--model_freqs FC FO ...`) or by specifying a file with freqs on each line (:code:`--model_freqs SOME_FILE`).

**model_invariant** : *list, default=["","IC","IO"]*
<br> Ways to treat invariant alignment columns. If calling from command line, these can be specified directly (:code:`--model_invariant IC IO ...`) or by specifying a file with invariants on each line (:code:`--model_invariant SOME_FILE`).

**restart** : *bool, default=False*
<br> Restart job from where it stopped in output directory. incompatible with overwrite.

**overwrite** : *bool, default=False*
<br> Whether or not to overwrite existing output. incompatible with restart.

**num_threads** : *int, default=-1*
<br> Number of threads to use. if -1 use all available.

**raxml_binary** : *str, optional*
<br> RAxML binary to use.

**generax_binary** : *str, optional*
<br> What generax binary to use.

---------- 


# 00. Infer the evolutionary model

The first step in a maximum likelihood phylogenetic analysis is determining the maximum likelihood model of sequence evolution. This includes the matrix for amino acid substitution (i.e., LG, JTT, WAG, etc.), the stationary frequencies for that model, rate variation parameters (𝚪 distribution, rate categories, etc.), and the proportion of invariant sites. Topiary uses a conventional method to find the best model (Abascal F, 2005). It uses RAxML-NG to generate a maximum parsimony tree from the alignment. It then optimizes branch lengths and other parameters using all 360 combinations of these model parameters implemented in the computational library that underlies RAxML-NG and GeneRax. Finally, it ranks these models based on a corrected Akaike Information Criterion, which penalizes models with excess parameters to prevent overfitting.

Although this protocol is done automatically, topiary returns a variety of statistics including AIC, AICc, and BIC to help users who want more control over model selection. Via the API, users can also specify a custom input tree or a subset of the models to test. (Note: as of the current version, topiary excludes the LG4M and LG4X models, as these cause GeneRax to crash during gene-species tree reconciliation).


### Arguments definitions and default parameters used in this step:

**df** : *pandas.DataFrame or str*
<br> Topiary data frame or csv written out from topiary df.

**starting_tree** : *str, optional*
<br> Tree in newick format. This will be used for the best model inference and starting tree for the maximum likelihood tree estimation. If not specified, the maximum parsimony tree is generated and used.

**model_matrices** : *list, default=["cpREV","Dayhoff","DCMut","DEN","Blosum62","FLU","HIVb","HIVw","JTT","JTT-DCMut","LG","mtART","mtMAM","mtREV","mtZOA","PMB","rtREV","stmtREV","VT","WAG"]*
<br> List of model matrices to check. If calling from command line, these can be specified directly (:code:`--model_matrices LG ...`) or by specifying a file with models on each line (:code:`--model_matrices SOME_FILE`).

**model_rates** : *list, default=["","G8"]*
<br> Ways to treat model rates. If calling from command line, these can be specified directly (:code:`--model_rates G8 ...`) or by specifying a file with rates on each line (:code:`--model_rates SOME_FILE`).

**model_freqs** : *list, default=["","FC","FO"]*
<br> Ways to treat model freqs. If calling from command line, these can be specified directly (:code:`--model_freqs FC FO ...`) or by specifying a file with freqs on each line (:code:`--model_freqs SOME_FILE`).

**model_invariant** : *list, default=["","IC","IO"]*
<br> Ways to treat invariant alignment columns. If calling from command line, these can be specified directly (:code:`--model_invariant IC IO ...`) or by specifying a file with invariants on each line (:code:`--model_invariant SOME_FILE`).

**calc_dir = output**
<br> Put the results of this calculation into a directory called "output." XX

**num_threads** : *int, default=-1*
<br> Number of threads to use. if -1 use all available.

**raxml_binary** : *str, optional*
<br> RAxML binary to use.

In [ ]:
# Find best phylogenetic model
topiary.find_best_model(df,
                        gene_tree=starting_tree,
                        model_matrices=model_matrices,
                        model_rates=model_rates,
                        model_freqs=model_freqs,
                        model_invariant=model_invariant,
                        calc_dir=output,
                        num_threads=num_threads,
                        raxml_binary=raxml_binary)
output = f"00_find-model"

# 01. Build a maximum likelihood gene tree

Topiary next infers an ML gene tree using the inferred phylogenetic model with the default RAxML-NG settings for the “--search” protocol. This starts the inference from 10 random trees and 10 different parsimony trees. It then optimizes the tree topology using an SPR subtree cutoff of 1, with an automatically selected fast versus slow SPR radius. Branch lengths are optimized using the NR-FAST algorithm. The tree with the highest likelihood is selected and used for downstream analyses. Advanced users have full access to all RAxML-NG options XXX.

### Arguments definitions and default parameters used in this step:

**prev_calculation = prev_calculation**
<br> Use the results of the previous calculation as input for this call. XX

**calc_dir = output** # XX "out_dir" in def?
<br> Put the results of this calculation into a directory called "output." XX

**num_threads** : *int, default=-1*
<br> Number of threads to use. If -1 use all available.

**raxml_binary** : *str, optional*
<br> RAxML binary to use.

**bootstrap** : *bool, default=False* # XX "no_bootstrap" in def?
<br> Do bootstrap replicates if set to True.

In [ ]:
# Generate the maximum likelihood tree without bootstraps
prev_calculation = output
topiary.generate_ml_tree(prev_calculation=prev_calculation,
                        calc_dir=output,
                        num_threads=num_threads,
                        raxml_binary=raxml_binary,
                        bootstrap=False)
output = f"01_ml-tree"

# 02. Reconcile gene and species trees

The next step in the pipeline is to reconcile the gene tree with the species tree. This automatically roots the tree and has been shown to improve the quality of reconstructed sequences (Groussin M, 2015). For this purpose, we use GeneRax, a new high-performance program for reconciling gene and species trees. Unlike other, heuristic, methods, GeneRax uses an explicit likelihood framework (Morel B, 2020). The final tree is the maximum likelihood estimate for an evolutionary model that includes both sequence evolution (i.e., LG) and evolutionary events (speciation, duplication, loss, and lateral gene transfer).

To do this, topiary uses the ML evolutionary model and ML gene tree inferred previously as inputs to GeneRax. For the rooted species tree, topiary automatically downloads the most recent synthetic tree from the Open Tree of Life (OTL) database (Rees J, 2017; Mctavish EJ, 2021). (Previous steps in the pipeline ensure that all sequences that have made it to this step come from species that are present in the OTL database). Any polytomies in this tree are resolved arbitrarily prior to the reconciliation inference. Topiary runs GeneRax with the default parameters (Morel B, 2020): topology optimization using rounds of subtree pruning and regrafting (SPR) with increasing radius (from 1 to 5) using the UndatedDTL reconciliation model. The UndatedDTL model accounts for duplication, transfer, and loss events. Topiary users can select the other implemented model— UndatedDL, which does not allow lateral transfer—however, the GeneRax authors strongly recommend using the UndatedDTL model.

The resulting tree is a maximum likelihood reconciled gene-species tree with optimized branch lengths and nodes labeled with inferred evolutionary events (speciation, duplication, or transfer). GeneRax returns a variety of other outputs that are made accessible to topiary users, but only the reconciled tree is used further in the pipeline.

### Arguments definitions and default parameters used in this step:

**prev_calculation = prev_calculation**
<br> Use the results of the previous calculation as input for this call. XX

**calc_dir = output** # XX "out_dir" in def?
<br> Put the results of this calculation into a directory called "output." XX

**allow_horizontal_transfer** : *bool, default=False* # XX "no_horizon..." in defintion?
<br> Whether to allow horizontal transfer during reconciliation. Default is to allow transfer (UndatedDTL; recommended). If flat set, use UndatedDL model.

**generax_binary** : *str, optional*
<br> What generax binary to use.

**num_threads** : *int, default=1*
<br> Number of threads to use. if -1 use all available.

**bootstrap** : *bool, default=False* # XX "no_bootstrap" in def?
<br> Do bootstrap replicates if set to True.

In [ ]:
# Reconcile without bootstrap.
prev_calculation = output

topiary.reconcile(prev_calculation=prev_calculation,
                  calc_dir=output,
                  allow_horizontal_transfer=allow_horizontal_transfer,
                  generax_binary=generax_binary,
                  num_threads=1, #num_threads, XXX HACK HACK HACK
                  bootstrap=False)

output = f"02_reconciliation"

# 03. Reconstruct ancestors

The next step is to infer sequences of ancestral nodes on the reconciled gene-species tree. For this, we use RAxML-NG, which implements a standard marginal ancestral reconstruction method (Yang Z, 1995). (This differs from previous versions of RAxML, which used a non- standard reconstruction method that was not comparable to other approaches). RAxML-NG finds the amino acid at each site in each ancestor that maximizes the likelihood of observing the sequence alignment given the tree, branch lengths, and phylogenetic model. This returns a matrix of posterior probabilities for each amino acid at each site in the alignment for each ancestral node. Topiary extracts the sequence of the maximum likelihood ancestor, as well as the so-called altAll version of the ancestor that incorporates alternate reconstructed amino acids at ambiguous positions. It uses a default cutoff of 0.25 to identify ambiguous sites (Eick GN, 2016); this can be set by the user.

The evolutionary models used by RAxML-NG do not explicitly treat gaps; therefore, the first draft of the reconstructed ancestor will be ungapped. Topiary assigns gaps by treating them as characters during ancestral character reconstruction (ACR). For this purpose, topiary uses the DOWNPASS (Maddison DR, 2000) algorithm as implemented by the PastML package (Ishikawa SA, 2019). The final output for this step consists of the gapped sequences of the maximum likelihood and altAll ancestors for each node. These have associated statistical supports: posterior probabilities for each reconstructed amino acid and support for gaps. Topiary also puts out a variety of summary graphs to help select high quality sequences.


### Arguments definitions and default parameters used in this step:

**prev_calculation = prev_calculation**
<br> Use the results of the previous calculation as input for this call. XX

**calc_dir = output** # XX "out_dir" in def?
<br> Put the results of this calculation into a directory called "output." XX

**num_threads** : *int, default=1*
<br> Number of threads to use. if -1 use all available.

**alt_cutoff** : *float, default=0.25*
<br> Cutoff to use for altAll alternate ancestral protein sequence generation. Should be between 0 and 1.

In [ ]:
# Generate ancestors
prev_calculation = output

topiary.generate_ancestors(prev_calculation=prev_calculation,
                          calc_dir=output,
                          num_threads=num_threads,
                          alt_cutoff=alt_cutoff)

output = f"03_ancestors"

# 04. Branch supports

To determine branch supports, topiary uses non-parametric bootstrapping (Felsenstein J, 1985). Briefly, RAxML-NG generates pseudoreplicate alignments by sampling columns, with replacement, from the input alignment. RAxML-NG then infers an evolutionary tree for each of these alignments. Topiary generates up to 1,000 bootstrap pseudoreplicates, using RAxML- NG’s automatic Extended Majority Rules (autoMRE) method with a cutoff of 0.03 to determine the exact number. The output from RAxML-NG is a collection of pseudoreplicate alignments and pseudoreplicate gene trees. Because we are reconstructing ancestors on the reconciled tree, we pass each pseudoreplicate alignment and gene tree into GeneRax for gene-species tree reconciliation, yielding a final collection of pseudoreplicate reconciled trees. Topiary then uses RAxML-NG to map these pseudoreplicate trees onto the ML reconciled tree as branch supports. Topiary also assesses convergence for the branch support estimate using the “--bsconverge” option.


### Arguments definitions and default parameters used in this step:

**prev_calculation = prev_calculation**
<br> Use the results of the previous calculation as input for this call. XX

**calc_dir = output** # XX "out_dir" in def?
<br> Put the results of this calculation into a directory called "output." XX

**num_threads** : *int, default=1*
<br> Number of threads to use. if -1 use all available.

**raxml_binary** : *str, optional*
<br> RAxML binary to use.

In [ ]:
# Generate bootstrap replicates for the tree
prev_calculation = output

topiary.generate_bootstraps(prev_calculation=prev_calculation,
                            calc_dir=output,
                            num_threads=num_threads,
                            raxml_binary=raxml_binary)

output = f"04_bootstraps"